# Problem:

- Toy Data 를 이용해서 gini criterion 을 이용해 best split 을 구하는 법을 알아봅니다.
- Section 1 에서는 'feature' 에 대해서 gini 계수를 계산하는 함수를 작성하고,
- Section 2 에서는 위에서 만든 함수로 gini 계수가 가장 높은 feature 를 찾는 get_attribute_gini_index 함수를 작성합니다.
- 마지막으로, Section 3 에서는 위에서 작성한 함수들을 이용해 2 개의 leaf node를 구한뒤, 각 node 의 best split feature 를 구합니다. 

# Section 1:

In [1]:
import pandas as pd 
import numpy as np

### Data Loading

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 순서 
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [3]:
df = pd_data
df[df['class_buys_computer'] =='no']

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
13,senior,medium,no,excellent,no


In [4]:
def get_gini(df, label):
    class_list = list(set(df[label]))
    freq_list = []
    for class_ in class_list:
        cnt = len(df[df[label] == str(class_)])
        freq_list.append(cnt)
    gini = 1
    total = df.shape[0]
    for i in freq_list:
        gini -= (i/total) **2 
    return gini

In [6]:
get_gini(pd_data, 'class_buys_computer')

0.4591836734693877

- get_gini 를 통해 label를 사용될 'class_buy_computer' feature 의 gini 계수는 0.459 라는 것을 알 수 있습니다. 
- 그러나 best spliting point 를 알기 위해서는 모든 feature 의 모든 이진분류된 attribute 의 gini 계수를 알아야합니다. 

# Section 2:
 - 가장 먼저 주어진 feature 의 모든 부분집합을 뱉어내는 함수가 필요합니다. 
 - 아래의 powerset 이 이 작업을 해줄 함수입니다. 이준걸님이 작성해주셨습니다.
 - ex) powerset({A,B,C}) -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

In [8]:
from itertools import chain, combinations

def powerset(feature_class):
    listed_data = list(feature_class)
    chain_set = chain.from_iterable(combinations(listed_data, i) 
                                    for i in range(len(listed_data)+1))
    return [set_data for set_data in chain_set]

In [9]:
powerset(pd_data.age.unique())

[(),
 ('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior'),
 ('youth', 'middle_aged', 'senior')]

### powerset() 아웃풋 중 중복되지 않는 attribute 을 선택
- powerset(pd_data.age) 는 3 가지 attribute 을 이용한 모든 부분집합을 구성합니다.
- 그러나, 우리는 {'youth'} 를 선택하게 되면 {'senior','middle_aged'} 는 필요치 않습니다.
- 따라서 get_binary_split 을 이용하여 필요한 attribtue group 만 선택합니다.

In [11]:
def get_binary_split(df, feature):
    result = []
    temp = powerset(df[feature].unique())
    total = len(df[feature].unique())
    for i in temp:
        if len(i) > 0 and len(i) != total :
            result.append(set(i))
    
    return result

In [12]:
get_binary_split(pd_data, "age")

[{'youth'},
 {'middle_aged'},
 {'senior'},
 {'middle_aged', 'youth'},
 {'senior', 'youth'},
 {'middle_aged', 'senior'}]

### 이제 get_binary_split 에서 얻은 attribute 들을 get_gini() 함수에 대입합니다.

In [16]:
def get_attribute_gini_index(df, attribute, label):
    result= dict()
    label_class = set(df[label].unique())
    binary_split = get_binary_split(df,attribute)
    attribute_class = set(df[attribute].unique())
    total_cnt = df.shape[0]
    
    for i in binary_split:
        temp_df = df.loc[df[attribute].isin(list(i))] 
        temp_df2 = df.loc[df[attribute].isin(list(attribute_class -i))]
        class_cnt = len(temp_df)
        class_cnt_diff = total_cnt - class_cnt
        gini = get_gini(temp_df,label)
        gini2 = get_gini(temp_df2,label)
        result_value = (class_cnt/total_cnt) * gini + (total_cnt-class_cnt)/total_cnt *gini2
        result_key = (',').join([j for j in i])
        result.update({result_key:result_value})

    return result

In [17]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'middle_aged,youth': 0.4571428571428572,
 'youth,senior': 0.35714285714285715,
 'middle_aged,senior': 0.3936507936507937}

- 이 중 가장 작은 값을 반환합니다:

In [20]:
result = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
for k, v in result.items():
    if v == min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").values()):
        print(k,v)
        break

middle_aged 0.35714285714285715


# Section 3:

#### get_attribute_gini 에서 얻은 output 에서 가장 작은 gini index 를 가지는 feature 와 class 를 반환

In [34]:
def find_min_gini(output):
    gini_value = [v for k,v in output.items()]
    features= [k for k,v in output.items()]
    idx = gini_value.index(min(gini_value))
    split_class = features[idx]
    split_class = split_class.split(',')
    split_feature = [i for i in pd_data.columns if i not in label][idx]
    return split_class, split_feature

In [45]:
label = "class_buys_computer"
output= {}
for feature in [i for i in pd_data.columns if i not in label]:
    temp = min(get_attribute_gini_index(pd_data,feature,label).items())
    output.update({str(temp[0]):temp[1]})
    


In [46]:
split_class, split_feature = find_min_gini(output)

In [49]:
output
#가장 낮은 split_class 는 'middle aged', 이는 'age' feature 에 해당된다. 

{'middle_aged': 0.35714285714285715,
 'high': 0.4428571428571429,
 'no': 0.3673469387755103,
 'excellent': 0.42857142857142855}

In [47]:
split_class, split_feature

(['middle_aged'], 'age')

In [38]:
print("Gini index of the feature '{}' is {:.4f}".format(split_feature,output[split_class[0]]))

Gini index of the feature 'age' is 0.3571


#### leaf_df1 : Label 분류가 완료되었으므로 이 leaf node 에서는 DT를 종료한다.

In [39]:
leaf_df1 = df[df[split_feature].isin(split_class)]
leaf_df1

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


#### leaf_df2: 위와 동일한 방식으로 best split 찾기

In [40]:
leaf_df2 = df[~df[split_feature].isin(split_class)]
leaf_df2

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


In [41]:
label = "class_buys_computer"
output= {}
for feature in [i for i in leaf_df2.columns if i not in label]:
    temp = min(get_attribute_gini_index(leaf_df2,feature,label).items())
    output.update({str(temp[0]):temp[1]})

In [43]:
output

{'senior': 0.48,
 'high': 0.375,
 'no': 0.31999999999999984,
 'excellent': 0.4166666666666667}

In [42]:
split_class, split_feature = find_min_gini(output)
split_class,split_feature
#Split_feature 는 'student'

(['no'], 'student')

In [44]:
print("Gini index of the feature '{}' is {:.4f}".format(split_feature,output[split_class[0]]))

Gini index of the feature 'student' is 0.3200
